# Processing LAPD arrests data

### Import Python tools and Jupyter configuration

In [18]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [19]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import json
import sqlite3
from datetime import datetime

In [20]:
import altair as alt
import altair_latimes as lat

alt.themes.register("latimes", lat.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('latimes')

In [21]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Download arrests data from LA City data portal

In [22]:
# 2010-2019 (https://data.lacity.org/Public-Safety/Arrest-Data-from-2020-to-Present/amvf-fr72)
!wget 'https://data.lacity.org/api/views/yru6-6re4/rows.csv?accessType=DOWNLOAD' -P '/Users/mhustiles/data/LAPD/'

In [23]:
!mv '/Users/mhustiles/data/LAPD/rows.csv?accessType=DOWNLOAD' '/Users/mhustiles/data/LAPD/Arrest_Data_from_2010_to_2019.csv'

In [24]:
# 2020-present (https://data.lacity.org/Public-Safety/Arrest-Data-from-2020-to-Present/amvf-fr72)
!wget 'https://data.lacity.org/api/views/amvf-fr72/rows.csv?accessType=DOWNLOAD' -P '/Users/mhustiles/data/LAPD/'

--2021-09-21 13:44:16--  https://data.lacity.org/api/views/amvf-fr72/rows.csv?accessType=DOWNLOAD
Resolving data.lacity.org (data.lacity.org)... 52.206.140.205, 52.206.68.26, 52.206.140.199
Connecting to data.lacity.org (data.lacity.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘/Users/mhustiles/data/LAPD/rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [        <=>         ]  29.04M  5.44MB/s    in 5.3s    

2021-09-21 13:44:22 (5.49 MB/s) - ‘/Users/mhustiles/data/LAPD/rows.csv?accessType=DOWNLOAD’ saved [30448445]



In [25]:
!mv '/Users/mhustiles/data/LAPD/rows.csv?accessType=DOWNLOAD' '/Users/mhustiles/data/LAPD/Arrest_Data_from_2020_to_Present.csv'

### Read the data

In [44]:
old = pd.read_csv(
    "/Users/mhustiles/data/LAPD/Arrest_Data_from_2010_to_2019.csv", low_memory=False
)

In [45]:
new = pd.read_csv(
    "/Users/mhustiles/data/LAPD/Arrest_Data_from_2020_to_Present.csv", low_memory=False
)

### Concatenate the old and the new

In [46]:
crimes = pd.concat([old, new])

In [47]:
crimes.columns = (
    crimes.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace("(", "", regex=False)
    .str.replace(")", "", regex=False)
    .str.replace("-", "_", regex=False)
)

In [48]:
crimes.head()

,report_id,report_type,arrest_date,time,area_id,area_name,reporting_district,age,sex_code,descent_code,charge_group_code,charge_group_description,arrest_type_code,charge,charge_description,disposition_description,address,cross_street,lat,lon,location,booking_date,booking_time,booking_location,booking_location_code
0,121920046,RFC,09/28/2012,930.0,19,Mission,1998,60,M,H,NaN,NaN,M,25620(A)BP,NaN,MISDEMEANOR COMPLAINT FILED,7600 WOODMAN AV,NaN,34.2111,-118.4309,POINT (-118.4309 34.2111),NaN,NaN,NaN,NaN
1,90712341,RFC,03/10/2011,940.0,7,Wilshire,776,45,M,B,18.0,Drunkeness,M,41.27CLAMC,DRINKING IN PUBLIC,MISDEMEANOR COMPLAINT FILED,4500 W WASHINGTON BL,NaN,34.0399,-118.3375,POINT (-118.3375 34.0399),NaN,NaN,NaN,NaN
2,121909585,RFC,04/01/2012,1715.0,19,Mission,1993,37,M,H,NaN,NaN,M,25620(A)BP,NaN,MISDEMEANOR COMPLAINT FILED,8100 SEPULVEDA PL,NaN,34.2208,-118.4662,POINT (-118.4662 34.2208),NaN,NaN,NaN,NaN
3,100109543,RFC,04/14/2010,1510.0,1,Central,158,38,F,W,13.0,Prostitution/Allied,M,653.22(A)PC,LOITER:INTENT:PROSTITUTION,MISDEMEANOR COMPLAINT FILED,7TH,TOWNE AV,34.0395,-118.2405,POINT (-118.2405 34.0395),NaN,NaN,NaN,NaN
4,100208368,RFC,03/18/2010,2110.0,2,Rampart,275,40,F,W,24.0,Miscellaneous Other Violations,M,103.107.1BL,ESCORT WITHOUT PERMIT,MISDEMEANOR COMPLAINT FILED,1900 W OLYMPIC BL,NaN,34.0508,-118.2777,POINT (-118.2777 34.0508),NaN,NaN,NaN,NaN


---

In [52]:
crimes.head()

,report_id,report_type,arrest_date,time,area_id,division_name,reporting_district,age,sex_code,descent_code,charge_group_code,charge_group_description,arrest_type_code,charge,charge_description,disposition_description,address,cross_street,latitude,longitide,address,booking_date,booking_time,booking_location,booking_location_code
0,121920046,RFC,09/28/2012,930.0,19,Mission,1998,60,M,H,NaN,NaN,M,25620(A)BP,NaN,MISDEMEANOR COMPLAINT FILED,7600 WOODMAN AV,NaN,34.2111,-118.4309,POINT (-118.4309 34.2111),NaN,NaN,NaN,NaN
1,90712341,RFC,03/10/2011,940.0,7,Wilshire,776,45,M,B,18.0,Drunkeness,M,41.27CLAMC,DRINKING IN PUBLIC,MISDEMEANOR COMPLAINT FILED,4500 W WASHINGTON BL,NaN,34.0399,-118.3375,POINT (-118.3375 34.0399),NaN,NaN,NaN,NaN
2,121909585,RFC,04/01/2012,1715.0,19,Mission,1993,37,M,H,NaN,NaN,M,25620(A)BP,NaN,MISDEMEANOR COMPLAINT FILED,8100 SEPULVEDA PL,NaN,34.2208,-118.4662,POINT (-118.4662 34.2208),NaN,NaN,NaN,NaN
3,100109543,RFC,04/14/2010,1510.0,1,Central,158,38,F,W,13.0,Prostitution/Allied,M,653.22(A)PC,LOITER:INTENT:PROSTITUTION,MISDEMEANOR COMPLAINT FILED,7TH,TOWNE AV,34.0395,-118.2405,POINT (-118.2405 34.0395),NaN,NaN,NaN,NaN
4,100208368,RFC,03/18/2010,2110.0,2,Rampart,275,40,F,W,24.0,Miscellaneous Other Violations,M,103.107.1BL,ESCORT WITHOUT PERMIT,MISDEMEANOR COMPLAINT FILED,1900 W OLYMPIC BL,NaN,34.0508,-118.2777,POINT (-118.2777 34.0508),NaN,NaN,NaN,NaN
